In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import panel as pn
import time
import random
import copy
import rtsvg
rt = rtsvg.RACETrack()

df1 = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk1.csv')
#df2 = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk2.csv')
#df3 = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk3.csv')
df  = pl.concat([df1]) # , df2, df3])

df = rt.columnsAreTimestamps(df, 'parsedDate')
df = df.rename({'TimeSeconds':                '_del1_',    'parsedDate':                 'timestamp',     'dateTimeStr':                '_del2_',
                'ipLayerProtocol':            'pro',       'ipLayerProtocolCode':        '_del3_',        'firstSeenSrcIp':             'sip',
                'firstSeenDestIp':            'dip',       'firstSeenSrcPort':           'spt',           'firstSeenDestPort':          'dpt',
                'moreFragments':              '_del4_',    'contFragments':              '_del5_',        'durationSeconds':            'dur',
                'firstSeenSrcPayloadBytes':   '_del6_',    'firstSeenDestPayloadBytes':  '_del7_',        'firstSeenSrcTotalBytes':     'soct',
                'firstSeenDestTotalBytes':    'doct',      'firstSeenSrcPacketCount':    'spkt',          'firstSeenDestPacketCount':   'dpkt',
                'recordForceOut':             '_del8_'})
df = df.drop(['_del1_', '_del2_', '_del3_', '_del4_', '_del5_', '_del6_', '_del7_', '_del8_'])
df = df.sample(1_000)
_relates_ = [('sip','dip')]
#_spi_ = rtsvg.RTSpreadLinesInteractivePanel(rt, df, {'relationships':_relates_, 'node_focus':'172.10.0.6'}, w=900, h=600)
#_spi_

_spec_    = {
             (0,0,1,1):('linkNode',    {'relationships':_relates_}),
             (1,0,1,1):('histogram',   {'bin_by':'sip'}),
             (2,0,4,1):('spreadLines', {'relationships':_relates_, 'node_focus':'172.10.0.6'})
            }
_ip_      = rt.interactivePanel(df, _spec_, 1000, 200)
#_ip_

_ln_params_ = {'relationships':_relates_, 'pos':{}, 'link_shape':'curve', 'link_size':'vary'}
_igl_       = rt.interactiveGraphPanel(df, _ln_params_, w=300, h=300)
_ln_params_ = {'relationships':_relates_, 'pos':{}, 'link_shape':'curve', 'link_size':'vary'}
_igl2_      = rt.interactiveGraphPanel(df, _ln_params_, w=300, h=300)
_spi_ = rtsvg.RTSpreadLinesInteractivePanel(rt, df, {'relationships':_relates_, 'node_focus':'172.10.0.6'}, w=800, h=200)
_histogram_ = rt.interactivePanel(df, {(0,0,1,1):('histogram', {'bin_by':'sip', 'count_by':'dip'})}, 200, 300)
_igl_.      register_companion_viz(_histogram_), _igl_.      register_companion_viz(_igl2_), _igl_.      register_companion_viz(_spi_)
_igl2_.     register_companion_viz(_histogram_), _igl2_.     register_companion_viz(_igl_),  _igl2_.     register_companion_viz(_spi_)
_histogram_.register_companion_viz(_igl_),       _histogram_.register_companion_viz(_igl2_), _histogram_.register_companion_viz(_spi_)
_spi_.      register_companion_viz(_igl_),       _spi_.      register_companion_viz(_igl2_), _spi_.      register_companion_viz(_histogram_)
pn.Column(_spi_, pn.Row(_igl_, _igl2_, _histogram_))

In [ ]:
#params_rt = {'df':df, 'relationships':_relates_, 'node_color':'node', 'every':'4h', 'h':384, 'w':1600}
#sl        = rt.spreadLines(node_focus='172.10.0.6', **params_rt)
#rt.tile([sl])

In [ ]:
params_rt = {'df':df, 'relationships':_relates_, 'node_color':'node', 'every':'4h', 'h':384, 'w':1600, 'include_svg_viewbox':False}
sl        = rt.spreadLines(node_focus='172.10.0.6', **params_rt)
sl._repr_svg_() # force a render to get the viewbox string
svg_wrap  = [f'<svg x="0" y="0" width="{sl.w}" height="{sl.h}" viewBox="{sl.viewBox()}">']
svg_wrap.append(sl._repr_svg_())
svg_wrap.append(f'<path d="{sl.__createPathDescriptionForAllEntities__()}" fill="#ff0000"/>')
svg_wrap.append('</svg>')
#rt.tile([''.join(svg_wrap)])

In [ ]:
svg_wrap  = [f'<svg x="0" y="0" width="{sl.w}" height="{sl.h}" viewBox="{sl.viewBox()}">']
svg_wrap.append(sl._repr_svg_())
svg_wrap.append(f'<path d="{sl.__createPathDescriptionOfSelectedEntities__(my_selection={'172.10.0.3'})}" fill="#ff0000"/>')
svg_wrap.append('</svg>')
#rt.tile([''.join(svg_wrap)])

In [ ]:
svg_wrap  = [f'<svg x="0" y="0" width="{sl.w}" height="{sl.h}" viewBox="{sl.viewBox()}">']
svg_wrap.append(sl._repr_svg_())
svg_wrap.append(f'<path d="{sl.__createPathDescriptionOfSelectedEntities__(my_selection={'172.10.0.3', '172.10.0.6'})}" fill="#ff0000"/>')
svg_wrap.append('</svg>')
#rt.tile([''.join(svg_wrap)])